## Step 1 : Load and Prepare The Dataset ??

In [1]:
from datasets import load_dataset

dataset = load_dataset("imdb")

In [2]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [3]:
dataset['train']

Dataset({
    features: ['text', 'label'],
    num_rows: 25000
})

In [4]:
dataset['train'][0]

{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

## Step 2 : Process The Data ??

In [5]:
from transformers import AutoTokenizer

# Load The Tokenizer ?
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

# Tokenize The Dataset ?
def tokenize_function (examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

In [6]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 50000
    })
})

In [7]:
tokenized_datasets['train'][0]

{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

## Step 3 : Set Up Training Data ??

In [8]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

training_args

TrainingArguments(
_n_gpu=0,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'non_blocking': False, 'gradient_accumulation_kwargs': None, 'use_configured_state': False},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
average_tokens_across_devices=False,
batch_eval_metrics=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_do_concat_batches=True,
eval_on_start=False,
eval_steps=None,
eval_strategy=IntervalStrategy.EPOCH,
eval_use_gather_object=False

## Step 4 : Initialize The Model ??

In [23]:
from transformers import AutoModelForSequenceClassification, Trainer

# Load The Pre-Trained Model
model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Initialize The Trainer ??
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test']
)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

C:\Users\hp\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\hp\.cache\huggingface\hub\models--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initi

## Step 6 : Train The Model ??

In [26]:
trainer.train()

KeyboardInterrupt: 

## Step 7 : Evaluate The Model ??

In [ ]:
results = trainer.evaluate()

print(results)

## Step 8 : Save The Fine-Tuned Model ??

In [ ]:
model.save_pretrained('./fine-tuned-model')

tokenizer.save_pretrained('./fine-tuned-tokenizer')

In [35]:
from transformers import pipeline
import arxiv
import pandas as pd

In [33]:
# Query To Fetch AI-Related Papers ??
query = 'ai OR artificial intelligence OR machine learning'

search = arxiv.Search(query=query, max_results=10, sort_by=arxiv.SortCriterion.SubmittedDate)

# Fetch Papers ??
papers = []

for result in search.results():
    papers.append({
        'published': result.published,
        'title': result.title,
        'abstract': result.summary,
        'categories': result.categories,
    })

# Convert To DataFrame ??
df = pd.DataFrame(papers)
pd.set_option('display.max_colwidth', None)
df.head(10)

C:\Users\hp\AppData\Local\Temp\ipykernel_8548\2565829634.py:9: DeprecationWarning: The 'Search.results' method is deprecated, use 'Client.results' instead
  for result in search.results():


,published,title,abstract,categories
0,2025-06-06 17:59:50+00:00,TerraFM: A Scalable Foundation Model for Unified Multisensor Earth Observation,"Modern Earth observation (EO) increasingly leverages deep learning to harness\nthe scale and diversity of satellite imagery across sensors and regions. While\nrecent foundation models have demonstrated promising generalization across EO\ntasks, many remain limited by the scale, geographical coverage, and spectral\ndiversity of their training data, factors critical for learning globally\ntransferable representations. In this work, we introduce TerraFM, a scalable\nself-supervised learning model that leverages globally distributed Sentinel-1\nand Sentinel-2 imagery, combined with large spatial tiles and land-cover aware\nsampling to enrich spatial and semantic coverage. By treating sensing\nmodalities as natural augmentations in our self-supervised approach, we unify\nradar and optical inputs via modality-specific patch embeddings and adaptive\ncross-attention fusion. Our training strategy integrates local-global\ncontrastive learning and introduces a dual-centering mechanism that\nincorporates class-frequency-aware regularization to address long-tailed\ndistributions in land cover.TerraFM achieves strong generalization on both\nclassification and segmentation tasks, outperforming prior models on GEO-Bench\nand Copernicus-Bench. Our code and pretrained models are publicly available at:\nhttps://github.com/mbzuai-oryx/TerraFM .",[cs.CV]
1,2025-06-06 17:59:28+00:00,Eigenspectrum Analysis of Neural Networks without Aspect Ratio Bias,"Diagnosing deep neural networks (DNNs) through the eigenspectrum of weight\nmatrices has been an active area of research in recent years. At a high level,\neigenspectrum analysis of DNNs involves measuring the heavytailness of the\nempirical spectral densities (ESD) of weight matrices. It provides insight into\nhow well a model is trained and can guide decisions on assigning better\nlayer-wise training hyperparameters. In this paper, we address a challenge\nassociated with such eigenspectrum methods: the impact of the aspect ratio of\nweight matrices on estimated heavytailness metrics. We demonstrate that\nmatrices of varying sizes (and aspect ratios) introduce a non-negligible bias\nin estimating heavytailness metrics, leading to inaccurate model diagnosis and\nlayer-wise hyperparameter assignment. To overcome this challenge, we propose\nFARMS (Fixed-Aspect-Ratio Matrix Subsampling), a method that normalizes the\nweight matrices by subsampling submatrices with a fixed aspect ratio. Instead\nof measuring the heavytailness of the original ESD, we measure the average ESD\nof these subsampled submatrices. We show that measuring the heavytailness of\nthese submatrices with the fixed aspect ratio can effectively mitigate the\naspect ratio bias. We validate our approach across various optimization\ntechniques and application domains that involve eigenspectrum analysis of\nweights, including image classification in computer vision (CV) models,\nscientific machine learning (SciML) model training, and large language model\n(LLM) pruning. Our results show that despite its simplicity, FARMS uniformly\nimproves the accuracy of eigenspectrum analysis while enabling more effective\nlayer-wise hyperparameter assignment in these application domains. In one of\nthe LLM pruning experiments, FARMS reduces the perplexity of the LLaMA-7B model\nby 17.3% when compared with the state-of-the-art method.","[cs.LG, cs.AI]"
2,2025-06-06 17:58:54+00:00,Distillation Robustifies Unlearning,"Current LLM unlearning methods are not robust: they can be reverted easily\nwith a few steps of finetuning. This is true even for the idealized unlearning\nmethod of training to imitate an oracle model that was never exposed to\nunwanted information, suggesting that output-based finetuning is insufficient\nto achieve robust unlearning. In a similar vein, we find that training a\nrandomly initialized stu

In [37]:
# Example abstract from API ??
abstract = df['abstract'][0]

summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

# Summarization ??
summarization_result = summarizer(abstract)

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

C:\Users\hp\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\hp\.cache\huggingface\hub\models--facebook--bart-large-cnn. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For be

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]


KeyboardInterrupt



In [ ]:
summarization_result[0]['summary_text']